In [4]:
from pandas_datareader import data as pdr
import yfinance as yf
import torch
import torch.nn as nn
import pandas as pd

In [15]:
yf.pdr_override()
y_symbols = ['BTC-USD']
from datetime import datetime
startdate = datetime(2015,1,1)
enddate = datetime(2022,12,31)
df = pdr.get_data_yahoo(y_symbols, start=startdate, end=enddate)

[*********************100%***********************]  1 of 1 completed


In [16]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-26,16842.250000,16920.123047,16812.369141,16919.804688,16919.804688,11886957804
2022-12-27,16919.291016,16959.845703,16642.072266,16717.173828,16717.173828,15748580239
2022-12-28,16716.400391,16768.169922,16497.556641,16552.572266,16552.572266,17005713920
2022-12-29,16552.322266,16651.755859,16508.683594,16642.341797,16642.341797,14472237479
2022-12-30,16641.330078,16643.427734,16408.474609,16602.585938,16602.585938,15929162910


In [18]:
len(df)

2921

In [19]:
# Define the number of previous days to use for prediction
num_prev_days = 2920

# Create a list of the close prices
close_prices = df['Close'].tolist()

# Create a list of tuples containing the previous num_prev_days close prices and the current close price
price_data = [(close_prices[i-num_prev_days:i], close_prices[i]) for i in range(num_prev_days, len(close_prices))]

# Split the data into input and target tensors
inputs = torch.tensor([i[0] for i in price_data], dtype=torch.float)
targets = torch.tensor([i[1] for i in price_data], dtype=torch.float)

# Define the neural network
class LLM(torch.nn.Module):
    def __init__(self, num_prev_days):
        super(LLM, self).__init__()
        self.linear1 = torch.nn.Linear(num_prev_days, 10)
        self.linear2 = torch.nn.Linear(10, 1)
        
    def forward(self, input):
        output = self.linear1(input)
        output = torch.relu(output)
        output = self.linear2(output)
        return output

# Instantiate the LLM model
model = LLM(num_prev_days)

# Define the loss function and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(500):
    # Forward pass
    predictions = model(inputs)
    
    # Compute loss
    loss = loss_fn(predictions, targets)
    
    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/500], Loss: {loss.item():.4f}')

# Use the trained model to make predictions for the next month
predictions = []
last_prices = close_prices[-num_prev_days:]

for i in range(1,31):
    input = torch.tensor(last_prices, dtype=torch.float)
    prediction = model(input).item()
    predictions.append(prediction)
    last_prices.pop(0)
    last_prices.append(prediction)

# Print the predictions
print(predictions)

Epoch [50/500], Loss: 1551095.1250
Epoch [100/500], Loss: 3293.4597
Epoch [150/500], Loss: 0.0022
Epoch [200/500], Loss: 0.4281
Epoch [250/500], Loss: 0.0011
Epoch [300/500], Loss: 0.0001
Epoch [350/500], Loss: 0.0000
Epoch [400/500], Loss: 0.0000
Epoch [450/500], Loss: 0.0000
Epoch [500/500], Loss: 0.0000
[16549.15234375, 16469.671875, 16110.638671875, 15825.5078125, 16070.4140625, 15777.7431640625, 15647.1611328125, 15596.3203125, 15607.42578125, 15780.890625, 15733.69140625, 15783.7412109375, 15967.1201171875, 15796.0341796875, 15870.755859375, 15687.6396484375, 15847.525390625, 15806.751953125, 15714.3515625, 15681.826171875, 15480.779296875, 15239.470703125, 15029.908203125, 14945.9228515625, 15023.080078125, 14937.427734375, 14801.3955078125, 14815.83203125, 14974.083984375, 14909.509765625]


/Users/gianmarcotucceri/miniconda/envs/mlp/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
